<a href="https://colab.research.google.com/github/mbribiescaucla-afk/imperial-valley-et/blob/main/ET_OO_A_DifModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. INSTALACIÓN Y AUTENTICACIÓN ---
!pip install -q earthengine-api geopandas

import geopandas as gpd
import pandas as pd
import ee
import json
import os
import time
from google.colab import drive

drive.mount('/content/drive')
ee.Authenticate()
ee.Initialize(project='ee-mbribiescaucla')

# --- 2. CARGAR SHAPEFILE ---
shapefile_path = '/content/drive/MyDrive/Shapefiles/CSB_IV_08_15.shp'
gdf = gpd.read_file(shapefile_path)

for y in range(2008, 2016):
    gdf[f'CDL{y}'] = pd.to_numeric(gdf[f'CDL{y}'], errors='coerce')

# --- 3. IDENTIFICAR PATRONES ---
pattern_results = []

for n_non36 in range(7, 0, -1):
    n_final36 = 8 - n_non36
    non36_years = [f'CDL{2008 + i}' for i in range(n_non36)]
    final36_years = [f'CDL{2008 + i}' for i in range(n_non36, 8)]
    pattern_label = 'O' * n_non36 + 'A' * n_final36
    required_cols = non36_years + final36_years
    gdf_clean = gdf.dropna(subset=required_cols)

    def match_pattern(row):
        if any(row[y] == 36 for y in non36_years): return False
        if any(row[y] != 36 for y in final36_years): return False
        return True

    matched = gdf_clean[gdf_clean.apply(match_pattern, axis=1)].copy()
    matched['pattern'] = pattern_label
    pattern_results.append(matched)

gdf_all = pd.concat(pattern_results)
print("Total de polígonos seleccionados:", len(gdf_all))

# --- 4. EXPORTAR GEOJSON A EE ---
gdf_all = gdf_all.to_crs("EPSG:4326")
geojson_path = '/content/all_patterns_to_alfalfa.json'
if os.path.exists(geojson_path): os.remove(geojson_path)
gdf_all.to_file(geojson_path, driver='GeoJSON')

with open(geojson_path) as f:
    geojson_data = json.load(f)

features = [ee.Feature(ee.Geometry(f['geometry']), f['properties']) for f in geojson_data['features']]
fc = ee.FeatureCollection(features)
print("Polígonos cargados en EE:", fc.size().getInfo())

# --- 5. CALCULAR ET DIARIA PROMEDIO PARA ENSEMBLE, DISALEXI, EEMETRIC ---
results_list = []

modelos = ['ensemble', 'disalexi', 'eemetric']
colecciones = {
    'ensemble': ('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'et_ensemble_sam'),
    'disalexi': ('OpenET/DISALEXI/CONUS/GRIDMET/MONTHLY/v2_0', 'et'),
    'eemetric': ('OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0', 'et')
}

for year in range(2008, 2016):
    print(f"🔄 Año {year}...")

    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    for model in modelos:
        print(f"  ➤ Modelo: {model.upper()}")
        collection_id, band_name = colecciones[model]
        col = ee.ImageCollection(collection_id).filterDate(start, end)
        image = col.select(band_name).sum()  # ET total anual en mm

        reduced = image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),
            scale=30
        ).map(lambda f: f.set('year', year)
                      .set('model', model.upper())
                      .set('et_mm_day', ee.Number(f.get('mean')).divide(365)))  # ET diaria promedio

        results_list.append(reduced)

# Combinar todos los resultados
all_results = ee.FeatureCollection(results_list).flatten()


# --- 6. EXPORTAR A DRIVE ---
export_desc = 'ET_daily_Ensemble_DisAlexi_eeMETRIC_2008_2015'
task = ee.batch.Export.table.toDrive(
    collection=all_results,
    description=export_desc,
    folder='Colab_ET',
    fileFormat='CSV'
)
task.start()

while task.active():
    print("Esperando exportación...")
    time.sleep(10)

print("✅ Exportación completada:", task.status())





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total de polígonos seleccionados: 2638
Polígonos cargados en EE: 2638
🔄 Año 2008...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2009...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2010...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2011...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2012...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2013...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2014...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
🔄 Año 2015...
  ➤ Modelo: ENSEMBLE
  ➤ Modelo: DISALEXI
  ➤ Modelo: EEMETRIC
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando exportación...
Esperando 

In [ ]:
df[['year', 'mean', 'et_mm_day_ensemble']].head(10)


,year,mean,et_mm_day_ensemble
0,2008,175.760611,0.481536
1,2008,287.851209,0.788633
2,2008,988.191760,2.707375
3,2008,180.899810,0.495616
4,2008,968.837885,2.654350
5,2008,976.808456,2.676188
6,2008,973.145571,2.666152
7,2008,322.708842,0.884134
8,2008,1035.800962,2.837811
9,2008,251.334471,0.688588


In [ ]:
import pandas as pd

# Ruta al archivo CSV exportado desde GEE
csv_path = '/content/drive/MyDrive/Colab_ET/ET_daily_Ensemble_DisAlexi_eeMETRIC_2008_2015.csv'
df = pd.read_csv(csv_path)

# Verificar columnas clave
required_cols = ['CSBID', 'year', 'model', 'et_mm_day']
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Falta la columna requerida: {col}")

# Crear tabla pivote: columnas serán ET [Modelo] [Año]
df_pivot = df.pivot_table(
    index='CSBID',
    columns=['model', 'year'],
    values='et_mm_day'
).reset_index()

# Renombrar columnas a formato: ET Ensemble 2008
df_pivot.columns = ['CSBID'] + [f"ET {model} {year}" for model, year in df_pivot.columns.tolist()[1:]]

# Extraer info base: patrón y clases CDL
cdl_cols = [f'CDL{y}' for y in range(2008, 2016)]
base_cols = ['CSBID', 'AreaRec', 'pattern'] + cdl_cols
df_base = df[base_cols].drop_duplicates(subset='CSBID')

# Unir todo y ordenar
df_final = pd.merge(df_base, df_pivot, on='CSBID', how='left')
et_cols = sorted([col for col in df_final.columns if col.startswith('ET ')])
df_final = df_final[['CSBID', 'AreaRec', 'pattern'] + cdl_cols + et_cols]

# Exportar a Excel
output_excel = '/content/drive/MyDrive/Colab_ET/ET_Patrones_Ensemble_DisAlexi_eeMETRIC.xlsx'
df_final.to_excel(output_excel, index=False)

print("✅ Excel generado:", output_excel)





✅ Excel generado: /content/drive/MyDrive/Colab_ET/ET_Patrones_Ensemble_DisAlexi_eeMETRIC.xlsx
